In [1]:
import functools
import re

import numpy as np
import json
import pandas as pd
from fuzzywuzzy import process

import auxiliar_4

/Users/erickavendanogarcia/Downloads/servicio_social/venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Encuesta

In [2]:
encuesta = pd.read_csv(r'datos/pareja-amigos.csv')
encuesta.columns = encuesta.iloc[1]
encuesta = encuesta.drop(encuesta.index[0:2])
encuesta = encuesta.reset_index(drop=True)

In [3]:
with open('../../encuestas/codigos_preguntas/propuesta.json', 'r', encoding='utf-8') as file:
    codigos_ocupaciones = json.load(file)

In [4]:
trabajos_oficiales = pd.Series(dtype='string')
for t in codigos_ocupaciones.values():
    trabajos_oficiales = pd.concat([trabajos_oficiales, pd.Series(t)])
trabajos_oficiales.reset_index(drop=True, inplace=True)
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [5]:
from auxiliar_4 import *

In [6]:
encuesta.head(3)

1,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,q786969,q786969_other,q67898,q67898_other,q67890,q67890_other,q3561,q3561_other,q532,q532_other
0,32,{question_not_shown},29/03/2021 11:56,1,es-MX,1382221824,29/03/2021 11:55,29/03/2021 11:56,29/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33,{question_not_shown},29/03/2021 18:49,1,es-MX,2028599215,29/03/2021 18:31,29/03/2021 18:49,29/03/2021 00:00,5299,...,A4,NaN,NaN,NaN,A2,NaN,A3,NaN,A2,NaN
2,34,{question_not_shown},29/03/2021 21:06,1,es-MX,1266979119,29/03/2021 21:05,29/03/2021 21:06,30/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Trabajo Pareja

## Limpieza Nivel 1

In [7]:
mostrar_unicos(encuesta,"parejaq0r1001")

0                    -oth-
1                       10
2                       11
3                       12
4                       16
5                       17
6                       18
7                       19
8                        2
9                       20
10                      21
11                      22
12                      24
13                       3
14                       4
15                       5
16                       6
17                       7
18                       8
19                       9
20    {question_not_shown}
Name: parejaq0r1001, dtype: object

In [8]:
mapeo_respuestas5 = {
    '2': "Actividades deportivas_1",
    '3': "Artista_2",
    '4': "Chofer_3",
    '5': "Comerciante sector formal_4",
    '6': "Comerciante sector informal_5",
    "7": "Empresario_6",
    "8": "Comercio Virtual_7",
    "9": "Atención a clientes_8",
    "10": "Administrativo_9",
    "11": "Gubernamental administrativo_10",
    "12": "Empleado gubernamental_11",
    "15": "Operador de maquinaria móvil_12",
    "16": "Personal técnico_13",
    "17": "Área educativa_14",
    "18": "Profesionista independiente_15",
    "19": "Seguridad_16",
    "20": "Área de la salud_17",
    "21": "Construcción_18",
    "22": "Fábrica_19",
    "23": "Agricultura_20",
    "24": "No trabaja/trabajaba_21"
}

In [9]:
llenar_respuesta_libre(encuesta, "parejaq0r1001", identificador_otro='-oth-', sufijo='_other')
llenar_vacios(encuesta, "parejaq0r1001", 'ninguna')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/Zoom4/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


In [10]:
encuesta['parejaq0r1001'] = encuesta['parejaq0r1001'].str.strip().str.lower()
encuesta['parejaq0r1001'] = encuesta['parejaq0r1001'].replace(mapeo_respuestas5)
mostrar_unicos(encuesta, "parejaq0r1001")

0            Actividades deportivas_1
1                    Administrativo_9
2                           Artista_2
3               Atención a clientes_8
4                            Chofer_3
5         Comerciante sector formal_4
6       Comerciante sector informal_5
7                  Comercio Virtual_7
8                     Construcción_18
9           Empleado gubernamental_11
10                       Empresario_6
11                         Fábrica_19
12    Gubernamental administrativo_10
13            No trabaja/trabajaba_21
14                Personal técnico_13
15     Profesionista independiente_15
16                       Seguridad_16
17                            auditor
18            cocinero en restaurante
19            coordinador territorial
20                         cordinador
21                  cuartos de escape
22                          diseñador
23                           empleado
24            empleado administrativo
25                            estudia
26          

## Limpieza de Nivel 2

### Columnas base

In [11]:
columnas_base5=['DEPORTE555','ART555','CHO5555', 'FOR555', 'INF555', 'EMPRE555', 'VIFT555', 'CLIE555','ADM555','EMPGU555', 'BUG55', 'MAQ555', 'TEC555', 'EDUC555', 'IND55', 'SEGU5', 'SALUD55', 'CONTR55', 'fabtr5', 'AGRICUL55']


In [12]:
len(columnas_base5)

20

In [13]:
encuesta[columnas_base5]

1,DEPORTE555,ART555,CHO5555,FOR555,INF555,EMPRE555,VIFT555,CLIE555,ADM555,EMPGU555,BUG55,MAQ555,TEC555,EDUC555,IND55,SEGU5,SALUD55,CONTR55,fabtr5,AGRICUL55
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},A2,{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},A2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_n

In [14]:
encuesta['DEPORTE555'] = encuesta['DEPORTE555'].replace({'A2': 'Entrenador de gimnasio_1','A3': 'Profesor de clases deportivas_2'})
encuesta['ART555'] = encuesta['ART555'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['CHO5555'] = encuesta['CHO5555'].replace({'A2': 'Taxista_1', 'A3': 'Uber_2','A4': 'Microbús_3', 'A5':'Camiones_4'})
encuesta['VIFT555'] = encuesta['VIFT555'].replace({'A2': 'Vendedor de productos por internet (mercado pago, amazon, instagram, etc.)_1'})
encuesta['INF555'] = encuesta['INF555'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3','A5':'Productos en un puesto no establecido'})

encuesta['FOR555'] = encuesta['FOR555'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})

encuesta['EMPRE555'] = encuesta['EMPRE555'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})
encuesta['CLIE555'] = encuesta['CLIE555'].replace({'A5': 'Promotor de seguros_4','A4':'Vendedor de tiendas departamentales_3','A2':'Atención a clientes_1','A3':'Recepcionista_2', 'A6':'Telefonía_5'})
encuesta['ADM555'] = encuesta['ADM555'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

encuesta['BUG55'] = encuesta['BUG55'].replace({'A2': 'Jefe de departamento_1','A3':'Jefe de área, subdirector_2','A4':'Subadministrador_3','A5':' Director_4','A6':'Administrador_5','A7':'Director general_6','A8':'Administrador general_7'})

encuesta['CONTR55'] = encuesta['CONTR55'].replace({'A2': 'Arquitecto_1','A3':'Ingeniero civil/albañil o trabajador general_2'})
encuesta['fabtr5'] = encuesta['fabtr5'].replace({'A2': 'Empleado_1'})

In [15]:
encuesta['SALUD55'] = encuesta['SALUD55'].replace(
    {'A2': 'Médico_1', 'A3': 'Enfermera_2', 'A4': 'Camillero_3', 'A5': 'Paramédico_4'})
encuesta['EDUC555'] = encuesta['EDUC555'].replace(
    {'A2': 'Investigador-profesor en universidad/preparatoria_1', 'A3': 'Educación básica_2',
     'A4': 'Técnico en educación (educador o guardería)_3'})
encuesta['IND55'] = encuesta['IND55'].replace(
    {'A2': 'Abogado_1', 'A3': 'Psicólogo_2', 'A4': 'Contador_3', 'A5': 'Dentista_4', 'A6': 'Médico independiente_5',
     'A7': ' Terapeuta_6'})
encuesta['SEGU5'] = encuesta['SEGU5'].replace(
    {'A2': 'Guardaespaldas_1', 'A3': 'Patrullero_2', 'A4': 'Vigilante_3', 'A5': 'Empleado de seguridad en bancos_4',
     'A6': 'Empleado de seguridad privada_5', 'A7': 'Policía de tránsito_6'})

In [16]:
encuesta['TEC555'] = encuesta['TEC555_comment']
encuesta['EMPGU555']=encuesta['EMPGU555_comment']


In [17]:
encuesta[columnas_base5]

1,DEPORTE555,ART555,CHO5555,FOR555,INF555,EMPRE555,VIFT555,CLIE555,ADM555,EMPGU555,BUG55,MAQ555,TEC555,EDUC555,IND55,SEGU5,SALUD55,CONTR55,fabtr5,AGRICUL55
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Empleado_1,{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Investigador-profesor en universidad/preparato...,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown

### Columnas con sufijos

In [18]:
columnas_sufijo5 = []
for col in columnas_base5:
    if col + '_other' in encuesta.columns:
        columnas_sufijo5.append(col + '_other')
    elif col + '_comment' in encuesta.columns:
        columnas_sufijo5.append(col + '_comment')
assert len(columnas_sufijo5) == len(columnas_base5)-1
columnas_sufijo5.sort()
columnas_sufijo5

['ADM555_other',
 'AGRICUL55_other',
 'ART555_other',
 'BUG55_other',
 'CHO5555_other',
 'CLIE555_other',
 'CONTR55_other',
 'DEPORTE555_other',
 'EDUC555_other',
 'EMPGU555_comment',
 'EMPRE555_other',
 'FOR555_other',
 'IND55_other',
 'INF555_other',
 'SALUD55_other',
 'SEGU5_other',
 'TEC555_comment',
 'VIFT555_other',
 'fabtr5_other']

In [19]:
len(columnas_sufijo5)

19

In [20]:
for col_con_sufijo in columnas_sufijo5:
    columna_sin_sufijo5 =  re.sub(r'_.*', '', col_con_sufijo)
    encuesta[columna_sin_sufijo5] = encuesta[columna_sin_sufijo5].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo5] = np.where(encuesta[columna_sin_sufijo5] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo5])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [21]:
encuesta[columnas_base5]

1,DEPORTE555,ART555,CHO5555,FOR555,INF555,EMPRE555,VIFT555,CLIE555,ADM555,EMPGU555,BUG55,MAQ555,TEC555,EDUC555,IND55,SEGU5,SALUD55,CONTR55,fabtr5,AGRICUL55
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Empleado_1,{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Investigador-profesor en universidad/preparato...,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown

In [22]:
encuesta

1,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,q786969,q786969_other,q67898,q67898_other,q67890,q67890_other,q3561,q3561_other,q532,q532_other
0,32,{question_not_shown},29/03/2021 11:56,1,es-MX,1382221824,29/03/2021 11:55,29/03/2021 11:56,29/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33,{question_not_shown},29/03/2021 18:49,1,es-MX,2028599215,29/03/2021 18:31,29/03/2021 18:49,29/03/2021 00:00,5299,...,A4,NaN,NaN,NaN,A2,NaN,A3,NaN,A2,NaN
2,34,{question_not_shown},29/03/2021 21:06,1,es-MX,1266979119,29/03/2021 21:05,29/03/2021 21:06,30/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,{question_not_shown},30/03/2021 12:57,1,es-MX,311980819,30/03/2021 12:46,30/03/2021 12:57,30/03/2021 00:00,5300,...,A3,NaN,A4,NaN,A3,NaN,A4,NaN,A2,NaN
4,36,{question_not_shown},30/03/2021 14:59,1,es-MX,1357322587,30/03/2021 14:46,30/03/2021 14:59,30/03/2021 00:00,5108,...,A5,NaN,A2,NaN,A2,NaN,A4,NaN,-oth-,NO SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,753,{question_not_shown},29/10/2022 17:57,1,es-MX,1624450592,29/10/2022 17:46,29/10/2022 17:57,29/10/2022 00:00,9027,...,A3,NaN,A2,NaN,A3,NaN,A5,NaN,A4,NaN
696,754,{question_not_shown},29/10/2022 18:41,1,es-MX,893888310,29/10/2022 18:35,29/10/2022 18:41,29/10/2022 00:00,9028,...,A4,NaN,A3,NaN,A5,NaN,A4,NaN,A2,NaN
697,755,{question_not_shown},29/10/2022 20:52,1,es-MX,977075814,29/10/2022 20:41,29/10/2022 20:52,30/10/2022 00:00,9029,...,A3,NaN,A3,NaN,A3,NaN,A4,NaN,A3,NaN
698,756,{question_not_shown},31/10/2022 19:50,1,es-MX,1847835502,31/10/2022 19:31,31/10/2022 19:50,31/10/2022 00:00,9030,...,A2,NaN,A2,NaN,A5,NaN,A5,NaN,A2,NaN


### Asignación de trabajo

In [23]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)

for columna in columnas_base5:
    encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['parejaq0r1001'] = encuesta['parejaq0r1001'].replace('{question_not_shown}', np.nan)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['parejaq0r1001'])
encuesta.drop(columns='parejaq0r1001', inplace=True)


encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/2082379804.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/2082379804.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)


0                                         abogado_1
1                                        acomodador
2                                   administrador_2
3                                        arquitecta
4                                        arquitecto
                          ...                      
88            vendedor de tiendas departamentales_3
89    vendedor por cuenta propia (negocio propio)_1
90                                           ventas
91                                      vigilante_3
92                                              NaN
Name: trabajo, Length: 93, dtype: object

In [24]:
encuesta['trabajo'] = encuesta['trabajo'].fillna('ninguna')

In [25]:
encuesta['trabajo']

0                                                ninguna
1                                                ninguna
2                                                ninguna
3                                                ninguna
4                                                ninguna
                             ...                        
695                                           empleado_1
696                                              ninguna
697    investigador-profesor en universidad/preparato...
698                                             estudiar
699                                              ninguna
Name: trabajo, Length: 700, dtype: object

In [26]:
trabajos_nuevos5 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos5

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/3895345309.py:1: SyntaxWarning: invalid escape sequence '\d'
  trabajos_nuevos5 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)


0                  estudiante
1                    juguetes
2                  cordinador
3                    finanzas
4                  estudiante
               ...           
90                   hospital
91    coordinador territorial
92    representante de ventas
93           empleado federal
94                   estudiar
Name: trabajo, Length: 95, dtype: object

### Correción de trabajos

In [27]:
from string_grouper import match_most_similar
correccion5 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos5,
                                 min_similarity=.40,
                                 ignore_index=True)
comparar5 = pd.DataFrame({'original': trabajos_nuevos5, 'correccion': correccion5})
comparar5['hubo_correccion'] = comparar5['original'].ne(comparar5['correccion'])
comparar5.sort_values(by='hubo_correccion', ascending=False)

/Users/erickavendanogarcia/Downloads/servicio_social/venv/lib/python3.12/site-packages/string_grouper/string_grouper.py:694: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  elif series_to_test.to_frame().applymap(


,original,correccion,hubo_correccion
47,ingeniero en minas,ingeniero,True
48,ingeniero en sistemas,sistemas,True
27,estudia,estudiante,True
28,estudiar,estudiante,True
33,asistente de camara,asistente,True
...,...,...,...
6,estudiante,estudiante,False
23,instructor,instructor,False
59,mesero,mesero,False
60,estudiante,estudiante,False


In [28]:
mapeo_correcciones5 = dict(zip(trabajos_nuevos5, correccion5))
mapeo_correcciones5

{'estudiante': 'estudiante',
 'juguetes': 'juguetes',
 'cordinador': 'cordinador',
 'finanzas': 'finanzas',
 'sanitizacion': 'sanitizacion',
 'ingeniero industrial': 'ingeniero',
 'estudia': 'estudiante',
 'ingeniería industrial': 'ingeniero',
 'asesorias educativas': 'asesor',
 'arquitecta': 'arquitecto',
 'productos': 'productos en un puesto no establecido',
 'ingeniero biomedico': 'ingeniero',
 'ventas': 'ventas',
 'ingeniero electricista': 'ingeniero',
 'diseñador': 'diseñador',
 'radio': 'radio',
 'tienda de abarrotes': 'abarrotes',
 'producciones musicales': 'producciones musicales',
 'instructor': 'instructor',
 'químico farmacobiólogo nutrición': 'nutriólogo',
 'innovacion tecnologica': 'innovacion tecnologica',
 'estudiante ingeniería': 'ingeniero',
 'estudiar': 'estudiante',
 'modelo': 'modelo',
 'relaciones laborales': 'relaciones laborales',
 'biologo': 'biologo',
 'bienes raices': 'bienes raices',
 'asistente de camara': 'asistente',
 'recursos humanos': 'recursos humanos'

In [29]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones5)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

0                                             abarrotes
1                                               abogado
2                                            acomodador
3                                         administrador
4                                            arquitecto
                            ...                        
69    vendedor de productos por internet (mercado pa...
70                  vendedor de tiendas departamentales
71          vendedor por cuenta propia (negocio propio)
72                                               ventas
73                                            vigilante
Name: trabajo, Length: 74, dtype: object

In [30]:
encuesta['trabajo']

0                                                ninguna
1                                                ninguna
2                                                ninguna
3                                                ninguna
4                                                ninguna
                             ...                        
695                                             empleado
696                                              ninguna
697    investigador-profesor en universidad/preparatoria
698                                           estudiante
699                                              ninguna
Name: trabajo, Length: 700, dtype: object

### Modificar algunos sinónimos

In [31]:
encuesta.iloc[532, encuesta.columns.get_loc('trabajo')] = 'empleado'
encuesta.iloc[403, encuesta.columns.get_loc('trabajo')] = 'empleado'
encuesta.iloc[46, encuesta.columns.get_loc('trabajo')] = 'coordinador'
encuesta.iloc[577, encuesta.columns.get_loc('trabajo')] ='financiero'
encuesta.iloc[54, encuesta.columns.get_loc('trabajo')] ='financiero'
encuesta.iloc[276, encuesta.columns.get_loc('trabajo')] ='biólogo'
encuesta.iloc[590, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[170, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[684, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[622, encuesta.columns.get_loc('trabajo')] ='ingeniero'
encuesta.iloc[468, encuesta.columns.get_loc('trabajo')] ='entrenador'
encuesta.iloc[488, encuesta.columns.get_loc('trabajo')] ='entrenador'
encuesta.iloc[499, encuesta.columns.get_loc('trabajo')] ='mesero'
encuesta.iloc[87, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[522, encuesta.columns.get_loc('trabajo')] ='trabaja en banco'
encuesta.iloc[42, encuesta.columns.get_loc('trabajo')] ='vendedor'

### Creación de categorías

In [32]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion coordinador es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion financiero es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es de

In [33]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0                  actividades deportivas_1
1     actividades elementales y de apoyo_20
2                          administrativo_9
3                                 artista_2
4                     atención a clientes_8
5                                  chofer_3
6               comerciante sector formal_5
7             comerciante sector informal_4
8                        comercio virtual_7
9                           construcción_17
10                           desconocido_99
11                empleado gubernamental_11
12                             empresario_6
13                               fábrica_18
14          gubernamental administrativo_10
15                               oficios_21
16           profesionista independiente_14
17                             seguridad_15
18                      área de la salud_16
19                        área educativa_13
dtype: string

In [34]:
categorias.value_counts()

desconocido_99                           565
área educativa_13                         31
profesionista independiente_14            14
fábrica_18                                13
actividades elementales y de apoyo_20     13
empleado gubernamental_11                 11
comerciante sector informal_4              8
atención a clientes_8                      6
actividades deportivas_1                   5
artista_2                                  5
construcción_17                            4
seguridad_15                               4
administrativo_9                           4
empresario_6                               3
comerciante sector formal_5                3
comercio virtual_7                         3
área de la salud_16                        3
chofer_3                                   2
gubernamental administrativo_10            2
oficios_21                                 1
Name: count, dtype: Int64

In [35]:
encuesta['trabajo'].value_counts()

trabajo
ninguna                      458
no trabaja/trabajaba          83
estudiante                    26
empleado                      13
administrador                 11
                            ... 
secundaria                     1
cuartos de escape              1
tese del estado de méxico      1
telefonía                      1
coordinador territorial        1
Name: count, Length: 68, dtype: int64

In [36]:
encuesta['categoria'] = categorias
encuesta[['trabajo', 'categoria']]

1,trabajo,categoria
0,ninguna,desconocido_99
1,ninguna,desconocido_99
2,ninguna,desconocido_99
3,ninguna,desconocido_99
4,ninguna,desconocido_99
...,...,...
695,empleado,fábrica_18
696,ninguna,desconocido_99
697,investigador-profesor en universidad/preparatoria,área educativa_13
698,estudiante,área educativa_13


### Asignación de la columna "Trabajas[Pareja]"
Esta columna se crea para corregir los datos de zoom 3, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [37]:
import pandas as pd

# Supongamos que tu DataFrame se llama 'encuesta' y ya está creado
encuesta['Trabajas[Pareja]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] in ['no trabaja/trabajaba', 'ninguna']:
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas[Pareja]'
        encuesta.at[index, 'Trabajas[Pareja]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas[Pareja]'
        encuesta.at[index, 'Trabajas[Pareja]'] = 'Y'

# Reordenar la lista de columnas para que 'Trabajas[Pareja]' esté antes de 'trabajo'
column_order = list(encuesta.columns)

# Mover 'Trabajas[Pareja]' antes de 'trabajo'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[Pareja]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]


In [38]:
 encuesta[['Trabajas[Pareja]','trabajo', 'categoria']]

1,Trabajas[Pareja],trabajo,categoria
0,N,ninguna,desconocido_99
1,N,ninguna,desconocido_99
2,N,ninguna,desconocido_99
3,N,ninguna,desconocido_99
4,N,ninguna,desconocido_99
...,...,...,...
695,Y,empleado,fábrica_18
696,N,ninguna,desconocido_99
697,Y,investigador-profesor en universidad/preparatoria,área educativa_13
698,Y,estudiante,área educativa_13


In [39]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías

In [40]:
encuesta.iloc[233, encuesta.columns.get_loc('trabajo')]="químico farmacobiólogo"

In [41]:
ocupaciones_formal = ["comerciante sector formal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","sobrerruedas","vendedor",'flores y productos en un puesto no establecido']
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [42]:
ocupaciones_of = ["cobranza","acomodador","mensajeria","petrolero","plomero","joyero","mecánico","armero", "bombero","ganadero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [43]:
ocupaciones_profin= ["historiador","químico farmacobiólogo","periodista","veterinario","financiero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"

In [44]:
ocupaciones_guber = ["empleado gubernamental", 'auditor', 'trabaja en delegación', "luz y fuerza","compañia de luz", "hacienda","jurisprudencia","presidente de la republica","diputado","bolsa de valores","tese del estado de méxico","coordinador","coordinador territorial"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"

In [45]:
ocupaciones_cons = ["contratista","ingeniero civil/albañil o trabajador general", "caminos y puentes"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cons), 'categoria'] = "construcción_17"

In [46]:
ocupaciones_saludd= ["optometrista","podológo","anestesiólogo","área de la salud","hospital","residente"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [47]:
ocupaciones_cliente= ["trabajador en cine","bienes raices","cuartos de escape"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"
ocupaciones_tec = ["instalación de cámaras"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"

In [48]:
ocupaciones_admin = ["trabaja en banco","relaciones laborales"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"
ocupaciones_art= ["producciones musicales"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_art), 'categoria'] = "artista_2"

In [49]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           544
área educativa_13                         31
profesionista independiente_14            17
empleado gubernamental_11                 15
fábrica_18                                13
actividades elementales y de apoyo_20     13
comerciante sector informal_4              9
atención a clientes_8                      8
construcción_17                            7
administrativo_9                           6
artista_2                                  6
actividades deportivas_1                   5
seguridad_15                               4
área de la salud_16                        4
oficios_21                                 4
comercio virtual_7                         3
empresario_6                               3
comerciante sector formal_5                3
chofer_3                                   2
gubernamental administrativo_10            2
personal técnico_12                        1
Name: count, dtype: Int64

In [50]:
encuesta[['trabajo', 'categoria']]

1,trabajo,categoria
0,ninguna,desconocido_99
1,ninguna,desconocido_99
2,ninguna,desconocido_99
3,ninguna,desconocido_99
4,ninguna,desconocido_99
...,...,...
695,empleado,fábrica_18
696,ninguna,desconocido_99
697,investigador-profesor en universidad/preparatoria,área educativa_13
698,estudiante,área educativa_13


### Resultado final

In [51]:
mapeo_codigos_categoria5 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria5[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria5

{'desconocido_99': 'A99',
 'artista_2': 'A02',
 'comercio virtual_7': 'A07',
 'área educativa_13': 'A13',
 'empleado gubernamental_11': 'A11',
 'comerciante sector informal_4': 'A04',
 'profesionista independiente_14': 'A14',
 'construcción_17': 'A17',
 'atención a clientes_8': 'A08',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'fábrica_18': 'A18',
 'empresario_6': 'A06',
 'actividades deportivas_1': 'A01',
 'comerciante sector formal_5': 'A05',
 'seguridad_15': 'A15',
 'chofer_3': 'A03',
 'área de la salud_16': 'A16',
 'gubernamental administrativo_10': 'A10',
 'oficios_21': 'A21'}

In [52]:
mapeo_codigos_categoria5['personal técnico_12'] = 'A12'
mapeo_codigos_categoria5

{'desconocido_99': 'A99',
 'artista_2': 'A02',
 'comercio virtual_7': 'A07',
 'área educativa_13': 'A13',
 'empleado gubernamental_11': 'A11',
 'comerciante sector informal_4': 'A04',
 'profesionista independiente_14': 'A14',
 'construcción_17': 'A17',
 'atención a clientes_8': 'A08',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'fábrica_18': 'A18',
 'empresario_6': 'A06',
 'actividades deportivas_1': 'A01',
 'comerciante sector formal_5': 'A05',
 'seguridad_15': 'A15',
 'chofer_3': 'A03',
 'área de la salud_16': 'A16',
 'gubernamental administrativo_10': 'A10',
 'oficios_21': 'A21',
 'personal técnico_12': 'A12'}

In [53]:
encuesta['categoria'].replace(mapeo_codigos_categoria5, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/42557719.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  encuesta['categoria'].replace(mapeo_codigos_categoria5, inplace=True)


0     A01
1     A02
2     A03
3     A04
4     A05
5     A06
6     A07
7     A08
8     A09
9     A10
10    A11
11    A12
12    A13
13    A14
14    A15
15    A16
16    A17
17    A18
18    A20
19    A21
20    A99
Name: categoria, dtype: string

In [54]:
mapeo_codigos_ocupaciones5 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria5[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones5[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones5

{'hace cinturones': 'A9901',
 'innovacion tecnologica': 'A9902',
 'ninguna': 'A9903',
 'radio': 'A9904',
 'diseñador': 'A0201',
 'fotógrafo': 'A0202',
 'modelo': 'A0203',
 'producciones musicales': 'A0204',
 'vendedor de productos por internet (mercado pago, amazon, instagram, etc.)': 'A0701',
 'educación básica': 'A1301',
 'estudiante': 'A1302',
 'investigador-profesor en universidad/preparatoria': 'A1303',
 'secundaria': 'A1304',
 'administrador': 'A1101',
 'auditor': 'A1102',
 'coordinador': 'A1103',
 'coordinador territorial': 'A1104',
 'tese del estado de méxico': 'A1105',
 'comerciante sector informal': 'A0401',
 'flores y productos en un puesto no establecido': 'A0402',
 'vendedor': 'A0403',
 'vendedor por cuenta propia (negocio propio)': 'A0404',
 'abogado': 'A1401',
 'biólogo': 'A1402',
 'financiero': 'A1403',
 'ingeniero': 'A1404',
 'químico farmacobiólogo': 'A1405',
 'arquitecto': 'A1701',
 'ingeniero civil/albañil o trabajador general': 'A1702',
 'atención a clientes': 'A08

In [55]:
mapeo_codigos_ocupaciones5["instalación de cámaras"] = "A1201"

In [56]:
encuesta['trabajo'].replace(mapeo_codigos_ocupaciones5,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/4216376514.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  encuesta['trabajo'].replace(mapeo_codigos_ocupaciones5,inplace=True)


0     A0101
1     A0102
2     A0103
3     A0201
4     A0202
      ...  
62    A2104
63    A9901
64    A9902
65    A9903
66    A9904
Name: trabajo, Length: 67, dtype: object

In [57]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[Pareja]', 'categoria': 'Categoria[Pareja]'})
encuesta[['Trabajas[Pareja]', 'Trabajo[Pareja]', 'Categoria[Pareja]']]

1,Trabajas[Pareja],Trabajo[Pareja],Categoria[Pareja]
0,N,A9903,A99
1,N,A9903,A99
2,N,A9903,A99
3,N,A9903,A99
4,N,A9903,A99
...,...,...,...
695,Y,A1801,A18
696,N,A9903,A99
697,Y,A1303,A13
698,Y,A1302,A13


In [58]:
#import json

#trabajos = { ... }

#with open("trabajos(pareja).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones5, archivo)

# Trabajo Amigo 2

## Limpieza Nivel 1

In [59]:
mostrar_unicos(encuesta,"Amigo2q0r134")

0                    -oth-
1                       10
2                       12
3                       16
4                       17
5                       18
6                       19
7                        2
8                       20
9                       21
10                      22
11                      24
12                       3
13                       4
14                       5
15                       6
16                       7
17                       9
18    {question_not_shown}
19                     NaN
Name: Amigo2q0r134, dtype: object

In [60]:
mapeo_respuestas6 = {
    '2': "Actividades deportivas_1",
    '3': "Artista_2",
    '4': "Chofer_3",
    '5': "Comerciante sector formal_4",
    '6': "Comerciante sector informal_5",
    "7": "Empresario_6",
    "8": "Comercio Virtual_7",
    "9": "Atención a clientes_8",
    "10": "Administrativo_9",
    "11": "Gubernamental administrativo_10",
    "12": "Empleado gubernamental_11",
    "15": "Operador de maquinaria móvil_12",
    "16": "Personal técnico_13",
    "17": "Área educativa_14",
    "18": "Profesionista independiente_15",
    "19": "Seguridad_16",
    "20": "Área de la salud_17",
    "21": "Construcción_18",
    "22": "Fábrica_19",
    "23": "Agricultura_20",
    "24": "No trabaja/trabajaba_21"
}

In [61]:
llenar_respuesta_libre(encuesta, "Amigo2q0r134", identificador_otro='-oth-', sufijo='_other')
llenar_vacios(encuesta, "Amigo2q0r134", 'ninguna')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/Zoom4/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


In [62]:
encuesta['Amigo2q0r134'] = encuesta['Amigo2q0r134'].str.strip().str.lower()
encuesta['Amigo2q0r134'] = encuesta['Amigo2q0r134'].replace(mapeo_respuestas5)
mostrar_unicos(encuesta, "Amigo2q0r134")

0               Actividades deportivas_1
1                       Administrativo_9
2                              Artista_2
3                  Atención a clientes_8
4                               Chofer_3
5            Comerciante sector formal_4
6          Comerciante sector informal_5
7                        Construcción_18
8              Empleado gubernamental_11
9                           Empresario_6
10                            Fábrica_19
11               No trabaja/trabajaba_21
12                   Personal técnico_13
13        Profesionista independiente_15
14                          Seguridad_16
15                               abogada
16       asesoria regularizacion escolar
17           asistente despacho abogados
18                      atiende farmacia
19                   brigadista forestal
20           caseta de cobro de un bazar
21                                  chef
22                        ciberseguridad
23                              cocinero
24              

## Limpieza Nivel 2

### Columnas Base

In [63]:
columnas_base6=[
    "DP999007",
    "ART56853A",
    "CHOF5559A",
    "FORMG66A",
    "INFOR4574MA",
    "EMPRESGHG5A",
    "VITUAL5G6A",
    "CLIENTESGFF5A",
    "ADMING575A",
    "EMPLGUB3646GA",
    "GUBERN465A",
    "MAQUIN4563A",
    "TECNICO456A",
    "EDUCAT465A",
    "INDE987A",
    "SEGURI3454A",
    "SALUD34546AQ",
    "CONSTR456A",
    "AGRICULDFERA",
    "fabrick"
]


In [64]:
len(columnas_base6)

20

In [65]:
encuesta[columnas_base6]

1,DP999007,ART56853A,CHOF5559A,FORMG66A,INFOR4574MA,EMPRESGHG5A,VITUAL5G6A,CLIENTESGFF5A,ADMING575A,EMPLGUB3646GA,GUBERN465A,MAQUIN4563A,TECNICO456A,EDUCAT465A,INDE987A,SEGURI3454A,SALUD34546AQ,CONSTR456A,AGRICULDFERA,fabrick
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},-oth-,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},A3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{

In [66]:
encuesta['DP999007'] = encuesta['DP999007'].replace(
    {'A2': 'Entrenador de gimnasio_1', 'A3': 'Profesor de clases deportivas_2'})
encuesta['ART56853A'] = encuesta['ART56853A'].replace(
    {'A2': 'Fotógrafo_1', 'A3': 'Pintor_2', 'A4': 'Músico_3', 'A5': 'Cantante_4'})
encuesta['CHOF5559A'] = encuesta['CHOF5559A'].replace(
    {'A2': 'Taxista_1', 'A3': 'Uber_2', 'A4': 'Microbús_3', 'A5': 'Camiones_4'})
encuesta['VITUAL5G6A'] = encuesta['VITUAL5G6A'].replace(
    {'A2': 'Vendedor de productos por internet (mercado pago, amazon, instagram, etc.)_1'})
encuesta['INFOR4574MA'] = encuesta['INFOR4574MA'].replace(
    {'A2': 'Vendedor de comida_1', 'A3': ' Periódicos_2', 'A4': 'Flores y productos en un puesto no establecido_3',
     'A5': 'Productos en un puesto no establecido'})

encuesta['FORMG66A'] = encuesta['FORMG66A'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})

encuesta['EMPRESGHG5A'] = encuesta['EMPRESGHG5A'].replace({'A2': 'Dueño_1', 'A3': 'Director o gerente_2'})
encuesta['CLIENTESGFF5A'] = encuesta['CLIENTESGFF5A'].replace(
    {'A5': 'Promotor de seguros_4', 'A4': 'Vendedor de tiendas departamentales_3', 'A2': 'Atención a clientes_1',
     'A3': 'Recepcionista_2', 'A6': 'Telefonía_5'})
encuesta['ADMING575A'] = encuesta['ADMING575A'].replace(
    {'A2': 'Contador público_1', 'A3': 'Administrador_2', 'A4': 'Recursos humanos_3'})

encuesta['GUBERN465A'] = encuesta['GUBERN465A'].replace(
    {'A2': 'Jefe de departamento_1', 'A3': 'Jefe de área, subdirector_2', 'A4': 'Subadministrador_3',
     'A5': ' Director_4', 'A6': 'Administrador_5', 'A7': 'Director general_6', 'A8': 'Administrador general_7'})

encuesta['CONSTR456A'] = encuesta['CONSTR456A'].replace(
    {'A2': 'Arquitecto_1', 'A3': 'Ingeniero civil/albañil o trabajador general_2'})
encuesta['fabrick'] = encuesta['fabrick'].replace({'A2': 'Empleado_1'})
encuesta['SALUD34546AQ'] = encuesta['SALUD34546AQ'].replace(
    {'A2': 'Médico_1', 'A3': 'Enfermera_2', 'A4': 'Camillero_3', 'A5': 'Paramédico_4'})
encuesta['EDUCAT465A'] = encuesta['EDUCAT465A'].replace(
    {'A2': 'Investigador-profesor en universidad/preparatoria_1', 'A3': 'Educación básica_2',
     'A4': 'Técnico en educación (educador o guardería)_3'})
encuesta['INDE987A'] = encuesta['INDE987A'].replace(
    {'A2': 'Abogado_1', 'A3': 'Psicólogo_2', 'A4': 'Contador_3', 'A5': 'Dentista_4', 'A6': 'Médico independiente_5',
     'A7': ' Terapeuta_6'})
encuesta['SEGURI3454A'] = encuesta['SEGURI3454A'].replace(
    {'A2': 'Guardaespaldas_1', 'A3': 'Patrullero_2', 'A4': 'Vigilante_3', 'A5': 'Empleado de seguridad en bancos_4',
     'A6': 'Empleado de seguridad privada_5', 'A7': 'Policía de tránsito_6'})
encuesta['TECNICO456A'] = encuesta['TECNICO456A_comment']
encuesta['EMPLGUB3646GA'] = encuesta['EMPLGUB3646GA_comment']

In [67]:
encuesta[columnas_base6]

1,DP999007,ART56853A,CHOF5559A,FORMG66A,INFOR4574MA,EMPRESGHG5A,VITUAL5G6A,CLIENTESGFF5A,ADMING575A,EMPLGUB3646GA,GUBERN465A,MAQUIN4563A,TECNICO456A,EDUCAT465A,INDE987A,SEGURI3454A,SALUD34546AQ,CONSTR456A,AGRICULDFERA,fabrick
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},-oth-,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Recepcionista_2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question

### Columnas con sufijos

In [68]:
columnas_sufijo6 = []
for col in columnas_base6:
    if col + '_other' in encuesta.columns:
        columnas_sufijo6.append(col + '_other')
    elif col + '_comment' in encuesta.columns:
        columnas_sufijo6.append(col + '_comment')
assert len(columnas_sufijo6) == len(columnas_base6)
columnas_sufijo6.sort()
columnas_sufijo6

['ADMING575A_other',
 'AGRICULDFERA_other',
 'ART56853A_other',
 'CHOF5559A_other',
 'CLIENTESGFF5A_other',
 'CONSTR456A_other',
 'DP999007_other',
 'EDUCAT465A_other',
 'EMPLGUB3646GA_comment',
 'EMPRESGHG5A_other',
 'FORMG66A_other',
 'GUBERN465A_other',
 'INDE987A_other',
 'INFOR4574MA_other',
 'MAQUIN4563A_comment',
 'SALUD34546AQ_other',
 'SEGURI3454A_other',
 'TECNICO456A_comment',
 'VITUAL5G6A_other',
 'fabrick_other']

In [69]:
len(columnas_sufijo6)

20

In [70]:
for col_con_sufijo in columnas_sufijo6:
    columna_sin_sufijo6 =  re.sub(r'_.*', '', col_con_sufijo)
    encuesta[columna_sin_sufijo6] = encuesta[columna_sin_sufijo6].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo6] = np.where(encuesta[columna_sin_sufijo6] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo6])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [71]:
encuesta[columnas_base6]

1,DP999007,ART56853A,CHOF5559A,FORMG66A,INFOR4574MA,EMPRESGHG5A,VITUAL5G6A,CLIENTESGFF5A,ADMING575A,EMPLGUB3646GA,GUBERN465A,MAQUIN4563A,TECNICO456A,EDUCAT465A,INDE987A,SEGURI3454A,SALUD34546AQ,CONSTR456A,AGRICULDFERA,fabrick
0,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
1,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
3,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
4,{question_not_shown},BAILARINA,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
696,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},Recepcionista_2,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
697,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown}
698,{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{question_not_shown},{ques

### Asignación de trabajo

In [72]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)

for columna in columnas_base6:
    encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['Amigo2q0r134'] = encuesta['Amigo2q0r134'].replace('{question_not_shown}', np.nan)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['Amigo2q0r134'])
encuesta.drop(columns='Amigo2q0r134', inplace=True)


encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/1304050975.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/1304050975.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  encuesta[columna] = encuesta[columna].replace('{question_not_shown}', np.nan)
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/1304050975.py:4: FutureWarning: Downcasting behavior in `repla

0                                            abogada
1                                          abogado_1
2                                         acomodador
3                                           actuaria
4                                           actuario
                           ...                      
120    vendedor por cuenta propia (negocio propio)_1
121                              ventas - publicidad
122                          ventas como dependiente
123                                     voluntariado
124                                              NaN
Name: trabajo, Length: 125, dtype: object

In [73]:
encuesta['trabajo'] = encuesta['trabajo'].fillna('ninguna')
encuesta['trabajo']

0                      ninguna
1                      no sabe
2                      ninguna
3      no trabaja/trabajaba_21
4                    bailarina
                ...           
695    no trabaja/trabajaba_21
696            recepcionista_2
697    no trabaja/trabajaba_21
698                   estudiar
699    no trabaja/trabajaba_21
Name: trabajo, Length: 700, dtype: object

In [74]:
trabajos_nuevos6 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos6

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/216671925.py:1: SyntaxWarning: invalid escape sequence '\d'
  trabajos_nuevos6 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)


0                                     no sabe
1                                   bailarina
2                                  estudiante
3                                  estudiante
4                                  preescolar
                        ...                  
164                     tiene varios trabajos
165                          educacion fisica
166    asistente de direccion en preparatoria
167                         estudia y trabaja
168                                  estudiar
Name: trabajo, Length: 169, dtype: object

### Correción de Trabajos

In [75]:
from string_grouper import match_most_similar
correccion6 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos6,
                                 min_similarity=.40,
                                 ignore_index=True)
comparar6 = pd.DataFrame({'original': trabajos_nuevos6, 'correccion': correccion6})
comparar6['hubo_correccion'] = comparar6['original'].ne(comparar6['correccion'])
comparar6.sort_values(by='hubo_correccion', ascending=False)

/Users/erickavendanogarcia/Downloads/servicio_social/venv/lib/python3.12/site-packages/string_grouper/string_grouper.py:694: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  elif series_to_test.to_frame().applymap(


,original,correccion,hubo_correccion
84,asistente dental,asistente,True
44,estudia,estudiante,True
97,estudiar,estudiante,True
94,promotor,promotor de seguros,True
91,estudiar,estudiante,True
...,...,...,...
79,asistente,asistente,False
78,mesero,mesero,False
27,idiomas,idiomas,False
28,mesero,mesero,False


In [76]:
mapeo_correcciones6 = dict(zip(trabajos_nuevos6, correccion6))
mapeo_correcciones6

{'no sabe': 'no sabe',
 'bailarina': 'bailarina',
 'estudiante': 'estudiante',
 'preescolar': 'preescolar',
 'caseta de cobro de un bazar': 'caseta de cobro de un bazar',
 'no se sabe': 'no se sabe',
 'estudia': 'estudiante',
 'hogar': 'hogar',
 'nutriologa': 'nutriólogo',
 'ejecutivo': 'ejecutivo',
 'ventas como dependiente': 'médico independiente',
 'no se': 'no se',
 'inventarios': 'inventarios',
 'idiomas': 'idiomas',
 'mesero': 'mesero',
 'ventas - publicidad': 'publicista',
 'inversiones': 'inversiones',
 'actuario': 'actuario',
 'eventos': 'eventos',
 'asistente despacho abogados': 'abogado',
 'cargador': 'cargador',
 'masajista': 'masajista',
 'asistente dental': 'asistente',
 'encuestador': 'encuestador',
 'estudiar': 'estudiante',
 'mesera': 'mesero',
 'brigadista forestal': 'brigadista forestal',
 'supervisor': 'supervisor',
 'ciberseguridad': 'ciberseguridad',
 'laboratorio': 'laboratorio',
 'actuaria': 'actuaria',
 'pasante': 'pasante',
 'criminologa': 'criminologa',
 'ven

In [77]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones6)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

0                                        abarrotes
1                                          abogado
2                                       acomodador
3                                         actuaria
4                                         actuario
                          ...                     
104                             vendedor de comida
105            vendedor de tiendas departamentales
106    vendedor por cuenta propia (negocio propio)
107                                   voluntariado
108                                 área educativa
Name: trabajo, Length: 109, dtype: object

In [78]:
encuesta['trabajo']

0                   ninguna
1                   no sabe
2                   ninguna
3      no trabaja/trabajaba
4                 bailarina
               ...         
695    no trabaja/trabajaba
696           recepcionista
697    no trabaja/trabajaba
698              estudiante
699    no trabaja/trabajaba
Name: trabajo, Length: 700, dtype: object

### Modificar algunos sinónimos

In [79]:
encuesta.iloc[684, encuesta.columns.get_loc('trabajo')] = 'asistente'
encuesta.iloc[609, encuesta.columns.get_loc('trabajo')] = 'empleado'
encuesta.iloc[42, encuesta.columns.get_loc('trabajo')] = 'cobrador'
encuesta.iloc[3, encuesta.columns.get_loc('trabajo')] ='financiero'
encuesta.iloc[233, encuesta.columns.get_loc('trabajo')] ='químico farmacobiólogo'
encuesta.iloc[620, encuesta.columns.get_loc('trabajo')] ='peluquero'
encuesta.iloc[365, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[437, encuesta.columns.get_loc('trabajo')] ='estudiante'
encuesta.iloc[694, encuesta.columns.get_loc('trabajo')] ='estudiante'
encuesta.iloc[415, encuesta.columns.get_loc('trabajo')] ='ingeniero'
encuesta.iloc[403, encuesta.columns.get_loc('trabajo')] ='gerente'
encuesta.iloc[523, encuesta.columns.get_loc('trabajo')] ='empleado'
encuesta.iloc[392, encuesta.columns.get_loc('trabajo')] ='idiomas'
encuesta.iloc[570, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[532, encuesta.columns.get_loc('trabajo')] ='oficinista'
encuesta.iloc[509, encuesta.columns.get_loc('trabajo')] ='ropa'
encuesta.iloc[390, encuesta.columns.get_loc('trabajo')] ='abarrotes'
encuesta.iloc[625, encuesta.columns.get_loc('trabajo')] ='natación'
encuesta.iloc[524, encuesta.columns.get_loc('trabajo')] ='fotógrafo'
encuesta.iloc[664, encuesta.columns.get_loc('trabajo')] ='abarrotes'
encuesta.iloc[77, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[653, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[557, encuesta.columns.get_loc('trabajo')] ='cocinero'
encuesta.iloc[677, encuesta.columns.get_loc('trabajo')] ='cocinero'
encuesta.iloc[425, encuesta.columns.get_loc('trabajo')] ='trabaja en banco'

### Creación de categorías

In [80]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion no sabe es desconocida
La ocupacion financiero es desconocida
La ocupacion bailarina es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion preescolar es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no trabaja/trabajaba es desconocida
La ocupacion no

In [81]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0                  actividades deportivas_1
1     actividades elementales y de apoyo_20
2                          administrativo_9
3                                 artista_2
4                     atención a clientes_8
5                                  chofer_3
6               comerciante sector formal_5
7             comerciante sector informal_4
8                           construcción_17
9                            desconocido_99
10                empleado gubernamental_11
11                             empresario_6
12                               fábrica_18
13                               oficios_21
14           profesionista independiente_14
15                      área de la salud_16
16                        área educativa_13
dtype: string

In [82]:
categorias.value_counts()

desconocido_99                           488
área educativa_13                         79
comerciante sector informal_4             22
actividades elementales y de apoyo_20     19
profesionista independiente_14            19
atención a clientes_8                     16
artista_2                                 10
fábrica_18                                 8
comerciante sector formal_5                8
oficios_21                                 6
área de la salud_16                        6
empleado gubernamental_11                  5
empresario_6                               5
administrativo_9                           4
construcción_17                            3
chofer_3                                   1
actividades deportivas_1                   1
Name: count, dtype: Int64

In [83]:
encuesta['trabajo'].value_counts()

trabajo
no trabaja/trabajaba                           324
ninguna                                        118
estudiante                                      65
vendedor por cuenta propia (negocio propio)     14
empleado                                         8
                                              ... 
pasante                                          1
criminologa                                      1
acomodador                                       1
corredora publica                                1
recepcionista                                    1
Name: count, Length: 98, dtype: int64

In [84]:
encuesta['categoria'] = categorias
encuesta[['trabajo', 'categoria']]

1,trabajo,categoria
0,ninguna,desconocido_99
1,no sabe,desconocido_99
2,ninguna,desconocido_99
3,financiero,desconocido_99
4,bailarina,desconocido_99
...,...,...
695,no trabaja/trabajaba,desconocido_99
696,recepcionista,atención a clientes_8
697,no trabaja/trabajaba,desconocido_99
698,estudiante,área educativa_13


### Asignación de la columna "Trabajas[Amigo2]"

In [85]:
import pandas as pd

# Supongamos que tu DataFrame se llama 'encuesta' y ya está creado
encuesta['Trabajas[Amigo2]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] in ['no trabaja/trabajaba', 'ninguna']:
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas[Pareja]'
        encuesta.at[index, 'Trabajas[Amigo2]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas[Pareja]'
        encuesta.at[index, 'Trabajas[Amigo2]'] = 'Y'

# Reordenar la lista de columnas para que 'Trabajas[Pareja]' esté antes de 'trabajo'
column_order = list(encuesta.columns)

# Mover 'Trabajas[Pareja]' antes de 'trabajo'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[Amigo2]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [86]:
 encuesta[['Trabajas[Amigo2]','trabajo', 'categoria']]

1,Trabajas[Amigo2],trabajo,categoria
0,N,ninguna,desconocido_99
1,Y,no sabe,desconocido_99
2,N,ninguna,desconocido_99
3,Y,financiero,desconocido_99
4,Y,bailarina,desconocido_99
...,...,...,...
695,N,no trabaja/trabajaba,desconocido_99
696,Y,recepcionista,atención a clientes_8
697,N,no trabaja/trabajaba,desconocido_99
698,Y,estudiante,área educativa_13


In [87]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías

In [88]:
encuesta.iloc[472, encuesta.columns.get_loc('trabajo')] ='empleado en seguridad privada'

In [89]:
ocupaciones_formal = ["comerciante sector formal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal", "sobrerruedas", "vendedor",
                        'flores y productos en un puesto no establecido']
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"
ocupaciones_of = ["cobrador", "acomodador", "masajista", "petrolero", "plomero", "joyero", "mecánico", "armero",
                  "bombero", "ganadero","lavaloza","electricista","inventarios","cargador","encuestador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"
ocupaciones_profin = ["historiador", "químico farmacobiólogo", "periodista", "veterinario", "financiero","economista","actuaria","fotoperiodista","criminologa","oficinista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"
ocupaciones_guber = ["empleado gubernamental", 'auditor', 'trabaja en delegación', "luz y fuerza", "compañia de luz",
                     "hacienda", "jurisprudencia", "presidente de la republica", "diputado", "bolsa de valores","cndh",
                     "tese del estado de méxico", "coordinador", "coordinador territorial","corredora publica","expide actas de defunción"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"
ocupaciones_cons = ["contratista", "ingeniero civil/albañil o trabajador general", "caminos y puentes"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cons), 'categoria'] = "construcción_17"
ocupaciones_saludd = ["optometrista", "podológo", "anestesiólogo", "área de la salud", "hospital", "residente"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"
ocupaciones_cliente = ["trabajador en cine", "bienes raices", "cuartos de escape"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"
ocupaciones_tec = ["instalación de cámaras"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"
ocupaciones_admin = ["trabaja en banco", "relaciones laborales","archivista","pasante"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"
ocupaciones_art = ["producciones musicales","bateria","ilustrador","bailarina"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_art), 'categoria'] = "artista_2"
ocupaciones_ar = ["trabajadores domésticos"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_ar), 'categoria'] = "actividades elementales y de apoyo_20"

In [90]:
ocupaciones_dep = ["natación"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_dep), 'categoria'] = "actividades deportivas_1"

ocupaciones_especificas = ["bibliotecario", "vicedirector", "pedagogía","preescolar","laboratorio"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

ocupaciones_empre = ["director o gerente","negocios","inversiones","ejecutivo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empre), 'categoria'] = "empresario_6"

ocupaciones_fabrica=["obrero","fábrica","supervisor"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_fabrica), 'categoria'] = "fábrica_18"

ocupaciones_seg = ["seguridad","militar","brigadista forestal","guardia","ciberseguridad","empleado en seguridad privada"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [91]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           452
área educativa_13                         81
profesionista independiente_14            26
comerciante sector informal_4             22
actividades elementales y de apoyo_20     19
atención a clientes_8                     16
oficios_21                                14
artista_2                                 13
fábrica_18                                 9
empleado gubernamental_11                  8
empresario_6                               8
comerciante sector formal_5                8
administrativo_9                           7
área de la salud_16                        6
seguridad_15                               4
construcción_17                            4
actividades deportivas_1                   2
chofer_3                                   1
Name: count, dtype: Int64

In [92]:
encuesta[['trabajo', 'categoria']]

1,trabajo,categoria
0,ninguna,desconocido_99
1,no sabe,desconocido_99
2,ninguna,desconocido_99
3,financiero,profesionista independiente_14
4,bailarina,artista_2
...,...,...
695,ninguna,desconocido_99
696,recepcionista,atención a clientes_8
697,ninguna,desconocido_99
698,estudiante,área educativa_13


### Resultado final

In [93]:
mapeo_codigos_categoria6 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria6[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria6

{'desconocido_99': 'A99',
 'área educativa_13': 'A13',
 'empresario_6': 'A06',
 'administrativo_9': 'A09',
 'área de la salud_16': 'A16',
 'comerciante sector formal_5': 'A05',
 'construcción_17': 'A17',
 'atención a clientes_8': 'A08',
 'empleado gubernamental_11': 'A11',
 'fábrica_18': 'A18',
 'comerciante sector informal_4': 'A04',
 'actividades elementales y de apoyo_20': 'A20',
 'chofer_3': 'A03',
 'artista_2': 'A02',
 'profesionista independiente_14': 'A14',
 'oficios_21': 'A21',
 'actividades deportivas_1': 'A01'}

In [94]:
mapeo_codigos_categoria6['seguridad_15'] = 'A15'
mapeo_codigos_categoria6

{'desconocido_99': 'A99',
 'área educativa_13': 'A13',
 'empresario_6': 'A06',
 'administrativo_9': 'A09',
 'área de la salud_16': 'A16',
 'comerciante sector formal_5': 'A05',
 'construcción_17': 'A17',
 'atención a clientes_8': 'A08',
 'empleado gubernamental_11': 'A11',
 'fábrica_18': 'A18',
 'comerciante sector informal_4': 'A04',
 'actividades elementales y de apoyo_20': 'A20',
 'chofer_3': 'A03',
 'artista_2': 'A02',
 'profesionista independiente_14': 'A14',
 'oficios_21': 'A21',
 'actividades deportivas_1': 'A01',
 'seguridad_15': 'A15'}

In [95]:
encuesta['categoria'].replace(mapeo_codigos_categoria6, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/2308444533.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  encuesta['categoria'].replace(mapeo_codigos_categoria6, inplace=True)


0     A01
1     A02
2     A03
3     A04
4     A05
5     A06
6     A08
7     A09
8     A11
9     A13
10    A14
11    A15
12    A16
13    A17
14    A18
15    A20
16    A21
17    A99
Name: categoria, dtype: string

In [96]:
mapeo_codigos_ocupaciones6 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria6[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones6[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones6

{'actuario': 'A9901',
 'eventos': 'A9902',
 'gestion': 'A9903',
 'ninguna': 'A9904',
 'no sabe': 'A9905',
 'no se': 'A9906',
 'no se sabe': 'A9907',
 'tiene varios trabajos': 'A9908',
 'voluntariado': 'A9909',
 'educación básica': 'A1301',
 'estudiante': 'A1302',
 'idiomas': 'A1303',
 'investigador-profesor en universidad/preparatoria': 'A1304',
 'laboratorio': 'A1305',
 'preescolar': 'A1306',
 'regularización': 'A1307',
 'área educativa': 'A1308',
 'director o gerente': 'A0601',
 'dueño': 'A0602',
 'ejecutivo': 'A0603',
 'gerente': 'A0604',
 'inversiones': 'A0605',
 'archivista': 'A0901',
 'contador público': 'A0902',
 'pasante': 'A0903',
 'recursos humanos': 'A0904',
 'trabaja en banco': 'A0905',
 'enfermera': 'A1601',
 'médico': 'A1602',
 'nutriólogo': 'A1603',
 'taquero': 'A0501',
 'vendedor de comida': 'A0502',
 'arquitecto': 'A1701',
 'ingeniero civil/albañil o trabajador general': 'A1702',
 'atención a clientes': 'A0801',
 'call center': 'A0802',
 'promotor de seguros': 'A0803',

In [97]:
mapeo_codigos_ocupaciones6["brigadista forestal"] = "A1501"
mapeo_codigos_ocupaciones6["ciberseguridad"] = "A1502"
mapeo_codigos_ocupaciones6["empleado en seguridad privada"] = "A1503"
mapeo_codigos_ocupaciones6["guardia"] = "A1504"
mapeo_codigos_ocupaciones6

{'actuario': 'A9901',
 'eventos': 'A9902',
 'gestion': 'A9903',
 'ninguna': 'A9904',
 'no sabe': 'A9905',
 'no se': 'A9906',
 'no se sabe': 'A9907',
 'tiene varios trabajos': 'A9908',
 'voluntariado': 'A9909',
 'educación básica': 'A1301',
 'estudiante': 'A1302',
 'idiomas': 'A1303',
 'investigador-profesor en universidad/preparatoria': 'A1304',
 'laboratorio': 'A1305',
 'preescolar': 'A1306',
 'regularización': 'A1307',
 'área educativa': 'A1308',
 'director o gerente': 'A0601',
 'dueño': 'A0602',
 'ejecutivo': 'A0603',
 'gerente': 'A0604',
 'inversiones': 'A0605',
 'archivista': 'A0901',
 'contador público': 'A0902',
 'pasante': 'A0903',
 'recursos humanos': 'A0904',
 'trabaja en banco': 'A0905',
 'enfermera': 'A1601',
 'médico': 'A1602',
 'nutriólogo': 'A1603',
 'taquero': 'A0501',
 'vendedor de comida': 'A0502',
 'arquitecto': 'A1701',
 'ingeniero civil/albañil o trabajador general': 'A1702',
 'atención a clientes': 'A0801',
 'call center': 'A0802',
 'promotor de seguros': 'A0803',

In [98]:
encuesta['trabajo'].replace(mapeo_codigos_ocupaciones6,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_7971/658387070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  encuesta['trabajo'].replace(mapeo_codigos_ocupaciones6,inplace=True)


0     A0101
1     A0102
2     A0201
3     A0202
4     A0203
      ...  
92    A9905
93    A9906
94    A9907
95    A9908
96    A9909
Name: trabajo, Length: 97, dtype: object

In [99]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[Amigo2]', 'categoria': 'Categoria[Amigo2]'})
encuesta[['Trabajas[Amigo2]', 'Trabajo[Amigo2]', 'Categoria[Amigo2]']]

1,Trabajas[Amigo2],Trabajo[Amigo2],Categoria[Amigo2]
0,N,A9904,A99
1,Y,A9905,A99
2,N,A9904,A99
3,Y,A1407,A14
4,Y,A0201,A02
...,...,...,...
695,N,A9904,A99
696,Y,A0804,A08
697,N,A9904,A99
698,Y,A1302,A13


In [100]:
#import json

#trabajos = { ... }

#with open("trabajos(amigo2).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones6, archivo)

# Diccionario de datos

Este diccionario contiene toda la información al respecto de las preguntas hechas en la encuesta de LimeSurvey, además de las descripciones a las preguntas, entre otras cosas

In [101]:
diccionario = pd.read_csv(r'datos/zoom4-dicc-tx.csv')
diccionario.head(3)

,item order,related,alive,new_code,code,question,options,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,1.0,G,_,fecha_encuesta,fecha,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,G,_,folio,folio,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,G,_,encuestador,encuestador,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
diccionario.columns

Index(['item order', 'related', 'alive', 'new_code', 'code', 'question',
       'options', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 6

In [103]:
diccionario_2 = diccionario.set_index('code', inplace=True)

In [104]:
mapeo_respuestas = auxiliar_4.construir_mapeo_preguntas_y_respuestas(diccionario_2)
mapeo_respuestas

AttributeError: 'NoneType' object has no attribute 'filter'

In [105]:
from auxiliar_4 import  mapear_codigos,llenar_vacios,llenar_respuesta_libre,mostrar_unicos,contar_faltantes,unificar_observaciones

# Funciones auxiliares

In [106]:
mapear_codigos = functools.partial(auxiliar_4.mapear_codigos, encuesta, mapeo_respuestas)
llenar_respuesta_libre = functools.partial(auxiliar_4.llenar_respuesta_libre, encuesta)
llenar_vacios = functools.partial(auxiliar_4.llenar_vacios, encuesta)
mostrar_unicos = functools.partial(auxiliar_4.mostrar_unicos, encuesta)
contar_faltantes = functools.partial(auxiliar_4.contar_faltantes, encuesta)
unificar_observaciones = functools.partial(auxiliar_4.unificar_observaciones, encuesta)

NameError: name 'mapeo_respuestas' is not defined

# Mapeo de enfermedades

Primero, de la encuesta, vamos a obtener todas las columnas que estén dentro de ese dataset, para luego poder filtrar y quedarnos con todas aquellas que contengan [other], y no pertenezcan a las ya mapeadas en el .json de edades

In [107]:
columnas = encuesta.columns.tolist()
columnas

['id',
 'token',
 'submitdate',
 'lastpage',
 'startlanguage',
 'seed',
 'startdate',
 'datestamp',
 'fecha',
 'folio',
 'encuestador',
 'encuestador_other',
 'pareja',
 'q0r587',
 'q0r938',
 'q0r938_other',
 'q0r219_SQ002',
 'q0r219_SQ003',
 'q0r219_SQ004',
 'q0r219_SQ005',
 'q0r219_other',
 'q0r632',
 'q0r632_other',
 'q0r69_SQ002',
 'q0r69_SQ003',
 'q0r69_SQ004',
 'q0r69_SQ005',
 'q0r69_other',
 'q0r192',
 'q0r192_other',
 'q0r914_SQ002',
 'q0r914_SQ003',
 'q0r914_SQ004',
 'q0r914_SQ005',
 'q0r914_other',
 'q0r120',
 'q0r120_other',
 'q0r262_77561',
 'q0r262_64740',
 'q0r262_78236',
 'q0r262_45719',
 'q0r262_other',
 'q0r934',
 'q0r934_other',
 'q0r850_SQ004',
 'q0r850_SQ005',
 'q0r850_SQ006',
 'q0r850_SQ007',
 'q0r850_other',
 'q0r701',
 'q0r701_other',
 'q0r649_SQ002',
 'q0r649_SQ003',
 'q0r649_SQ004',
 'q0r649_SQ005',
 'q0r649_other',
 'q0r325',
 'q0r325_other',
 'q0r458_SQ002',
 'q0r458_SQ003',
 'q0r458_SQ005',
 'q0r458_SQ006',
 'q0r458_SQ007',
 'q0r458_SQ008',
 'q0r458_SQ009',


In [108]:
columnas_filtradas = [col for col in columnas if '_other' in col]
columnas_filtradas

['encuestador_other',
 'q0r938_other',
 'q0r219_other',
 'q0r632_other',
 'q0r69_other',
 'q0r192_other',
 'q0r914_other',
 'q0r120_other',
 'q0r262_other',
 'q0r934_other',
 'q0r850_other',
 'q0r701_other',
 'q0r649_other',
 'q0r325_other',
 'q0r458_other',
 'q0r60_other',
 'q0r60_othercomment',
 'q0r439_other',
 'q0r808_other',
 'q0r339_other',
 'q0r507_other',
 'q0r529_other',
 'q0r645_other',
 'q0r6_other',
 'q0r603_other',
 'q0r343_other',
 'q0r338_other',
 'q0r630_other',
 'q0r179_other',
 'q0r578_other',
 'q0r97_other',
 'q0r421_other',
 'q0r751_other',
 'q0r751_othercomment',
 'q0r789_other',
 'Amigo1Q2345_other',
 'DPORT88_other',
 'ARTIST88_other',
 'CHOFER88_other',
 'FORMAL888_other',
 'INFOR555_other',
 'EMPRE888_other',
 'VIRTUAL888_other',
 'ATENCION555_other',
 'ADMIN588_other',
 'GUB588_other',
 'EMPLEGUB55_other',
 'EDUCA556_other',
 'INDP56_other',
 'SEGURI544_other',
 'SALUD564_other',
 'DCONSTU56_other',
 'AFGRU56_other',
 'fabrica328_other',
 'q24_other',
 'q85678

## Estudios

In [109]:
columnas_estudio = diccionario[diccionario['new_code'].str.contains('estudio', case=False, na=False)]
# Crear un diccionario que mapee las columnas "code" y "new_code" para estas columnas específicas relacionadas con el trabajo
diccionario_estudio = dict(zip(columnas_estudio['code'], columnas_estudio['new_code']))
# Mostrar algunos elementos del diccionario para verificar el mapeo
list(diccionario_estudio.items())[:10]

KeyError: 'code'

In [ ]:
encuesta_estudios = ['q0r458_other', 'q0r421_other']

In [ ]:
columnas_filtradas_2 = [col for col in columnas_filtradas if col in encuesta_estudios]
columnas_filtradas_2

In [ ]:
# Obtener los valores de las columnas filtradas
valores_columnas = encuesta[columnas_filtradas_2].values.flatten().tolist()

# Eliminar valores nulos o vacíos
valores_columnas = [valor for valor in valores_columnas if pd.notnull(valor) and valor != '']

# Convertir los valores a minúsculas
valores_columnas = [valor.lower() for valor in valores_columnas]

# Filtrar para eliminar '{question_not_shown}'
valores_columnas = [valor for valor in valores_columnas if valor != '{question_not_shown}' and valor not in columnas_filtradas_2]

# Obtener valores únicos
estudios_unicos = list(set(valores_columnas))

print("Lista original de estudios:")
print(estudios_unicos)

In [ ]:
estudios_generales = [
    "Orina (análisis de orina)",
    "Tomografía computarizada (TC)",
    "Electroencefalograma (EEG)",
    "Radiografía de tórax",
    "Estudios de tiroides",
    "Prueba de antígeno prostático específico (PSA)",
    "Ultrasonido renal (USG renal)",
    "Cultivo de heces",
    "Biometría hemática completa",
    "Colesterol total, LDL, HDL",
    "Prueba de tolerancia a la glucosa",
    "Mamografía",
    "Densitometría ósea",
    "Colonoscopia",
    "Prueba de Papanicolaou",
    "Electrocardiograma (ECG)",
    "Ultrasonido abdominal",
    "Ultrasonido pélvico",
    "Vitamina D",
    "Ácido úrico",
    "Perfil hepático",
    "Perfil renal",
    "Perfil lipídico",
    "Resonancia magnética (RM)"
]


In [ ]:
def encontrar_relacion_fuzzy(estudio):
    match, score = process.extractOne(estudio, estudios_generales)
    # Puedes ajustar el umbral de puntuación según tus necesidades
    if score > 80:
        return match
    return ''

df_relacion_fuzzy = pd.DataFrame({
    'clave': estudios_unicos,
    'valor': [encontrar_relacion_fuzzy(estudio) for estudio in estudios_unicos]
})

# Mostrar el resultado
df_relacion_fuzzy


In [ ]:
df_relacion_fuzzy.to_excel(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom4\output\lista_estudios.xlsx", index=False)

## Enfermedades

In [ ]:
encuesta_enfermedades = ['q0r751_other', 'q0r86_other']
columnas_filtradas_2 = [col for col in columnas_filtradas if col in encuesta_enfermedades]
columnas_filtradas_2

In [ ]:
# Obtener los valores de las columnas filtradas
valores_columnas = encuesta[columnas_filtradas_2].values.flatten().tolist()

# Eliminar valores nulos o vacíos
valores_columnas = [valor for valor in valores_columnas if pd.notnull(valor) and valor != '']

# Convertir los valores a minúsculas
valores_columnas = [valor.lower() for valor in valores_columnas]

# Filtrar para eliminar '{question_not_shown}'
valores_columnas = [valor for valor in valores_columnas if valor != '{question_not_shown}' and valor not in columnas_filtradas_2]

# Obtener valores únicos
enfermedades_unicas = list(set(valores_columnas))

print("Lista original de enfermedades:")
print(enfermedades_unicas)

In [110]:
enfermedades_estandar = [
    "Sobrepeso u obesidad (Dob)","Diabetes (Ddi)","Prediabetes (Dpd)","Síndrome metabólico (Dsm)","Hipertensión (Dh)","Colesterol alto (Dc)",
    "Triglicéridos altos (Dtr)","Cáncer (Dcan)","Algún problema cardiovascular (Dcar)","Algún problema respiratorio (Dres)","COVID19 (Dcov)",
    "Alcoholismo (Dal)","Tabaquismo (Dta)","Trastorno de la alimentación (bulimia, anorexia) (Dba)","Trastorno de atracón (Datr)",
    "Ansiedad (Dan)","Depresión (Dde)","Trastorno obsesivo compulsivo (Dtoc)",'Síndrome del ovario poliquístico (SOP)',
    'Alergias','Alzheimer','Apendicitis','Artritis','Cataratas','Anemia','Cirrosis hepática','Epilepsia','Esclerosis múltiple',
    'Esquizofrenia','Gastritis','Glaucoma','Hepatitis','Herpes','Hipertiroidismo','Hipotiroidismo','Insuficiencia renal']

In [111]:
def encontrar_relacion_fuzzy(enfermedad):
    match, score = process.extractOne(enfermedad, enfermedades_estandar)
    # Puedes ajustar el umbral de puntuación según tus necesidades
    if score > 80:
        return match
    return ''

In [112]:
df_relacion_enfermedades = pd.DataFrame({
    'clave': enfermedades_unicas,
    'valor': [encontrar_relacion_fuzzy(enfermedad) for enfermedad in enfermedades_unicas]
})

# Mostrar el resultado
df_relacion_enfermedades

NameError: name 'enfermedades_unicas' is not defined

In [113]:
df_relacion_enfermedades.to_excel(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom4\output\lista_enfermedades.xlsx", index=False)

NameError: name 'df_relacion_enfermedades' is not defined

# Mapeo de edades

In [114]:
# Cargando el archivo JSON
with open('C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\Zoom2\\output\\edades.json', 'r') as file:
    age_ranges_dict = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\Zoom2\\output\\edades.json'

In [115]:
# Identificar las columnas [other] que se relacionan con las edades en la encuesta
age_related_other_columns = [col for col in encuesta.columns if '[other]' in col and col.split('[')[0] in age_ranges_dict]

In [116]:
# Verificando los primeros elementos de age_ranges_dict y las primeras columnas de survey_df
age_ranges_dict[:5], list(encuesta.columns)[:20]

NameError: name 'age_ranges_dict' is not defined

In [117]:
# Ajustando age_ranges_dict para ser un diccionario plano en lugar de una lista de diccionarios
flat_age_ranges_dict = {}
for entry in age_ranges_dict:
    flat_age_ranges_dict.update(entry)

# Verificando la estructura del diccionario ajustado
dict(list(flat_age_ranges_dict.items())[:5])

NameError: name 'age_ranges_dict' is not defined

In [118]:
# Función para verificar si un valor es numérico
def is_numeric(val):
    if isinstance(val, (int, float)):
        return True
    if isinstance(val, str) and val.replace('.', '', 1).isdigit():
        return True
    return False

In [119]:
# Eliminar columnas [other] que no tienen datos y corregir las respuestas no numéricas
columns_not_found = []  # Lista para guardar columnas que no se encuentran
for col in age_related_other_columns:
    try:
        # Si todos los valores en la columna son NaN, eliminar la columna
        if encuesta[col].isna().all():
            encuesta.drop(columns=[col], inplace=True)
        else:
            # Reemplazar valores no numéricos con 99
            encuesta[col] = encuesta[col].apply(lambda x: x if is_numeric(x) else 99)
    except KeyError:
        columns_not_found.append(col)

columns_not_found

[]

In [120]:
def extract_min_max_from_string(s):
    """
    Extracts minimum and maximum values from a string using regex.
    """
    numbers = re.findall(r'(\d+\.*\d*)', s)
    if len(numbers) == 1:
        return float(numbers[0]), float(numbers[0])
    elif len(numbers) > 1:
        return float(numbers[0]), float(numbers[1])
    return None, None

In [121]:
def regex_map_age_to_correct_column(row):
    for question_key, age_ranges in flat_age_ranges_dict.items():
        other_column = f"{question_key}[other]"
        
        # If the [other] column is not present or its value is NaN, continue with the next iteration
        if other_column not in row or pd.isna(row[other_column]):
            continue

        age_value_str = str(row[other_column])
        age_min, age_max = extract_min_max_from_string(age_value_str)

        # If no valid age values are found, continue with the next iteration
        if age_min is None or age_max is None:
            continue

        # Map the age range to the correct columns
        for age_range in age_ranges:
            defined_age_min, defined_age_max = age_range['min'], age_range['max']
            var = age_range['var']
            
            if (defined_age_min <= age_min <= defined_age_max) or (defined_age_min <= age_max <= defined_age_max):
                correct_column = f"{question_key}[{var}]"
                row[correct_column] = "Y"

    return row

In [122]:
# Apply the regex-based function to the DataFrame
regex_corrected_survey_df = encuesta.apply(regex_map_age_to_correct_column, axis=1)

In [123]:
# Mostrando las primeras filas del DataFrame corregido
regex_corrected_survey_df.head()

1,id,token,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,...,q3561,q3561_other,q532,q532_other,Trabajas[Pareja],Trabajo[Pareja],Categoria[Pareja],Trabajas[Amigo2],Trabajo[Amigo2],Categoria[Amigo2]
0,32,{question_not_shown},29/03/2021 11:56,1,es-MX,1382221824,29/03/2021 11:55,29/03/2021 11:56,29/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,N,A9903,A99,N,A9904,A99
1,33,{question_not_shown},29/03/2021 18:49,1,es-MX,2028599215,29/03/2021 18:31,29/03/2021 18:49,29/03/2021 00:00,5299,...,A3,NaN,A2,NaN,N,A9903,A99,Y,A9905,A99
2,34,{question_not_shown},29/03/2021 21:06,1,es-MX,1266979119,29/03/2021 21:05,29/03/2021 21:06,30/03/2021 00:00,5298,...,NaN,NaN,NaN,NaN,N,A9903,A99,N,A9904,A99
3,35,{question_not_shown},30/03/2021 12:57,1,es-MX,311980819,30/03/2021 12:46,30/03/2021 12:57,30/03/2021 00:00,5300,...,A4,NaN,A2,NaN,N,A9903,A99,Y,A1407,A14
4,36,{question_not_shown},30/03/2021 14:59,1,es-MX,1357322587,30/03/2021 14:46,30/03/2021 14:59,30/03/2021 00:00,5108,...,A4,NaN,-oth-,NO SE,N,A9903,A99,Y,A0201,A02


In [124]:
#regex_corrected_survey_df.to_csv(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom2\output\encuesta_limpia.csv", index=False)

# Mapeo de trabajos

Filtraremos todas las columnas bases 

In [125]:
columnas_trabajo = traductor[traductor['new_code'].str.contains('trabajo', case=False, na=False)]
# Crear un diccionario que mapee las columnas "code" y "new_code" para estas columnas específicas relacionadas con el trabajo
diccionario_trabajo = dict(zip(columnas_trabajo['code'], columnas_trabajo['new_code']))
# Mostrar algunos elementos del diccionario para verificar el mapeo
list(diccionario_trabajo.items())[:10]

NameError: name 'traductor' is not defined

In [126]:
# Modificar las claves del diccionario de trabajo para reflejar la diferencia en la nomenclatura
diccionario_trabajo_modificado = {key.replace('_other', '[other]'): value for key, value in diccionario_trabajo.items()}

# Seleccionar solo las columnas que existen en la encuesta y en el diccionario de trabajo modificado
columnas_existentes_modificadas = [col for col in diccionario_trabajo_modificado.keys() if col in encuesta.columns]
columnas_existentes_modificadas

NameError: name 'diccionario_trabajo' is not defined

In [127]:
# Crear un DataFrame para almacenar las respuestas relacionadas con el trabajo
respuestas_trabajo_df = encuesta[columnas_existentes_modificadas]

NameError: name 'columnas_existentes_modificadas' is not defined

In [128]:
# Renombrar las columnas usando el diccionario de trabajo modificado
respuestas_trabajo_df.columns = respuestas_trabajo_df.columns.map(lambda col: diccionario_trabajo_modificado.get(col, col))

# Limpiar las respuestas, eliminando los espacios en blanco y convirtiendo a minúsculas
respuestas_trabajo_df = respuestas_trabajo_df.apply(lambda x: x.str.strip().str.lower())

# Llenar los valores vacíos con 'ninguna'
respuestas_trabajo_df = respuestas_trabajo_df.fillna('ninguna')

NameError: name 'respuestas_trabajo_df' is not defined

In [129]:
respuestas_trabajo_df

NameError: name 'respuestas_trabajo_df' is not defined

In [130]:
for idx, columna in enumerate(respuestas_trabajo_df.columns):
    valores_unicos = respuestas_trabajo_df.iloc[:, idx].astype(str).unique()
    print(f"Los valores únicos en la columna '{columna}' (índice {idx}) son: {valores_unicos}")

NameError: name 'respuestas_trabajo_df' is not defined

Agregamos el JSON códigos de ocupaciones

In [131]:
with open('C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\codigos_preguntas\\codigos_ocupaciones.json', 'r', encoding='utf-8') as file:
    codigos_ocupaciones = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Elvira Magallanes\\Documents\\Servicio Social\\encuestas\\codigos_preguntas\\codigos_ocupaciones.json'

In [132]:
from difflib import get_close_matches

# Filtrar las columnas [other] relacionadas con trabajos para cada familiar
columnas_other_familiares = [col for col in encuesta.columns if any(relative in col for relative in relatives) and "_other" in col]

# Mapear ocupaciones escritas incorrectamente a las ocupaciones correctas
for col in columnas_other_familiares:
    for idx, value in encuesta[col].dropna().iteritems():
        # Buscar coincidencias cercanas en el archivo JSON de códigos de ocupaciones
        matches = get_close_matches(value, codigos_ocupaciones.values(), n=1, cutoff=0.6)
        if matches:
            encuesta.at[idx, 'trabajo'] = list(codigos_ocupaciones.keys())[list(codigos_ocupaciones.values()).index(matches[0])]

# Mostrar las primeras filas de la columna 'trabajo' después del mapeo
encuesta['trabajo'].head()

NameError: name 'relatives' is not defined